In [ ]:
import torch
import torch.optim as optim
from tqdm import tqdm
from model import BiLSTM_CRF
from utils import prepare

In [ ]:
hparams = {
    'epochs': 5,
    'batch_size': 10,
    'embedding_dim': 300,
    'hidden_dim': 150
}

In [ ]:
tag2idx, vocab, loader = prepare(hparams)
hparams['vocab_size'] = len(vocab)
hparams['seq_length'] = loader.dataset.max_length

In [ ]:
model = BiLSTM_CRF(hparams, tag2idx)

In [ ]:
with torch.no_grad():
    record = next(iter(loader))
    print(record['sentence'])
    print(model(record), record['label'])

In [ ]:
optimizer = optim.SGD(model.parameters(),lr=0.001)
for epoch in range(hparams['epochs']):
    tqdm_ = tqdm(enumerate(loader))
    for step,x in tqdm_:
        model.zero_grad()
        loss = model.neg_log_likelihood(x)
        loss.backward()
        optimizer.step()

        tqdm_.set_description("epoch {:d} , step {:d} , loss: {:.4f}".format(epoch+1, step, loss.item()))